In [84]:
import pandas as pd
import urllib.request

In [85]:
# SpotiPy and Spotify API

import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.oauth2 as oauth2

CLIENT_ID = "###############"
CLIENT_SECRET = "##################"

spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET))

In [86]:
# prepare the data
spotify_songs = pd.read_excel("spotify.xlsx", sheet_name="all")
spotify_songs["title"] = spotify_songs["title"].astype(str)
spotify_songs_sorted = spotify_songs.sort_values("title", key=lambda col: col.str.lower()).reset_index()
spotify_songs_artist_sorted = spotify_songs.sort_values("artist", key=lambda col: col.str.lower()).reset_index()

In [ ]:
# test
track = "death bed (coffee for your head)"
artist = "Powfu"
res = spotify.search(q='artist:' + artist + ' track:' + track, type='track', limit=1)

In [ ]:
# song pics
title_length = len(spotify_songs['title'])
sequence = ["%.3d" % i for i in range(title_length)]

for i in range(title_length):
    track = spotify_songs['title'][i]
    artist = spotify_songs['artist'][i]
    res = spotify.search(q='artist:' + artist + ' track:' + track, type='track', limit=1)
    album_image_url = res['tracks']['items'][0]['album']['images'][0]['url']
    file_location = "./album-covers/" + sequence[i] + ".png"
    urllib.request.urlretrieve(album_image_url, file_location)

In [ ]:
# lyrics

import numpy as np
import lyricsgenius
from requests.exceptions import Timeout

title_length = len(spotify_songs['title'])
lyrics = []

genius = lyricsgenius.Genius("#################################")
for i in range(title_length):
    track = spotify_songs['title'][i]
    artist = spotify_songs['artist'][i]
    retries = 0
    while retries < 3:
        try:
            song = genius.search_song(track, artist)
        except Timeout as e:
            retries += 1
            continue
        if song is not None:
            lyrics.append(song.lyrics)
        else:
            lyrics.append(np.NAN)
        break

In [101]:
# artist pics

unique_artists = spotify_songs_artist_sorted['artist'].drop_duplicates().reset_index()

artist_names = []
artist_image_urls = []
artist_followers = []
artist_popularitys = []

for i in range(len(unique_artists)):
    artist = unique_artists['artist'][i]
    res = spotify.search(q='artist:' + artist, type='artist', limit=1)
    artist_image_url = res['artists']['items'][0]['images'][0]['url']
    artist_follower = res['artists']['items'][0]['followers']['total']
    artist_popularity = res['artists']['items'][0]['popularity']
    artist_names.append(artist)
    artist_image_urls.append(artist_image_url)
    artist_followers.append(artist_follower)
    artist_popularitys.append(artist_popularity)

In [107]:
# artisit info

artist_info = pd.DataFrame({'artist_names':artist_names, 'artist_followers':artist_followers,
 'artist_popularitys':artist_popularitys, "artist_image_urls":artist_image_urls})

In [ ]:
# downloader

import urllib.request
length_artist = len(artist_info['artist_image_urls'])
for i in range(length_artist):
    sequence = ["%.3d" % i for i in range(length_artist)]
    image_url = artist_info['artist_image_urls'][i]
    file_location = "./artist-pics/" + sequence[i] + ".png"
    urllib.request.urlretrieve(image_url, file_location)

In [112]:
# crop the artist pic to round

from PIL import Image, ImageChops, ImageDraw
length_artist = len(artist_info['artist_image_urls'])
sequence = ["%.3d" % i for i in range(length_artist)]
for i in range(length_artist):
    file_location = "./artist-pics/" + sequence[i] + ".png"
    im = Image.open(file_location)

    im = im.resize((200,200), Image.ANTIALIAS)
    bigsize = (im.size[0] * 3, im.size[1] * 3)
    mask = Image.new('L', bigsize, 0)
    draw = ImageDraw.Draw(mask)
    draw.ellipse((0, 0) + bigsize, fill=255)
    mask = mask.resize(im.size, Image.ANTIALIAS)
    im.putalpha(mask)

    im.save(file_location)